In [1]:
import pandas as pd
import numpy as np

# --- 1. Load Data ---
data = pd.read_excel(
    "real-expenditure.xlsx",
    sheet_name = 'ABMI',
    index_col = 0,
    na_values = ['#N/A'],
    keep_default_na = True,
    header = 3
    )

lag = 4

# --- 2. Index & Column Cleanup ---
data.index.name = 'Date'
# Cleans the index format from 'YYYY:Q' to 'YYYYQ' and converts to PeriodIndex with quarterly frequency
data.index = pd.to_datetime(data.index, format='%b-%y').to_period('Q')
try:
    data.columns = pd.to_datetime(data.columns)
except:
    data.columns = pd.to_datetime(data.columns.astype(str).str.split().str[0], errors='coerce')
display(data)

# --- 3. Real-Time Calculation Function ---
## Calculates the real-time YoY growth rate by pulling both the first reported RGDP value
## and the revised RGDP value from the same vintage column.
def calculate_real_time_RGDP_yoy(df_levels, lag):
    real_time_vintage_cols = df_levels.apply(pd.Series.first_valid_index, axis = 1)
    real_time_yoy = pd.Series(index = df_levels.index, dtype = float)

    for obs_date in df_levels.index[lag:]:

        vintage_col_name = real_time_vintage_cols.loc[obs_date]

        if pd.isna(vintage_col_name):
            continue

        ## Get the full time time series of RGDP levels available on that trading date.
        vintage_series = df_levels[vintage_col_name]

        ## Exract numerator (t) and denomicator (t - 4) from the SAME vintage column.
        numerator_level = vintage_series.loc[obs_date]
        denominatorlevel = vintage_series.loc[obs_date - lag]
        if not pd.isna(numerator_level) and not pd.isna(denominatorlevel):
            yoy = ((numerator_level / denominatorlevel) -1) * 100
            real_time_yoy.loc[obs_date] = yoy

    return real_time_yoy

# --- 4. Execute All Necessary Calculations ---
final_revised_RGDP = data.iloc[:, -1]
final_revised_RGDP_yoy = ((final_revised_RGDP / final_revised_RGDP.shift(lag)) - 1) * 100
real_time_RGDP_yoy = calculate_real_time_RGDP_yoy(data, lag)
acceleration_signal = real_time_RGDP_yoy -  real_time_RGDP_yoy.shift(4)

# --- 5. Create Final Output DataFrame ---
final_output_df = pd.DataFrame({
    'Final_Revised_RGDP': final_revised_RGDP.values,
    'Final_Revised_RGDP_YoY_Growth': final_revised_RGDP_yoy.values,
    'Real_Time_YoY_Growth': real_time_RGDP_yoy,
    'Acceleration Signal': acceleration_signal
})

final_output_df.index.name = 'Date'
final_output_df = final_output_df.dropna().reset_index(drop = False).set_index('Date', drop = True)

display(final_output_df)
final_output_df.to_excel("Final_RGDP_YoY_Output(UK).xlsx")


/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_36746/829039164.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data.columns = pd.to_datetime(data.columns)


,1990-01-01,1990-02-01,1990-03-01,1990-04-01,1990-05-01,1990-06-01,1990-07-01,1990-08-01,1990-09-01,1990-10-01,...,2015-12-01,2016-01-01,2016-02-01,2016-03-01,2016-04-01,2016-05-01,2016-06-01,2016-07-01,2016-08-01,2016-09-01
Date,,,,,,,,,,,,,,,,,,,,,
1970Q1,65128.0,65128.0,65128.0,65128.0,65128.0,65128.0,65128.0,65144.0,65144.0,65144.0,...,160727.0,160727.0,160727.0,160727.0,160727.0,160727.0,165838.0,165838.0,165838.0,165838.0
1970Q2,66625.0,66625.0,66625.0,66625.0,66625.0,66625.0,66625.0,66642.0,66642.0,66642.0,...,164547.0,164547.0,164547.0,164547.0,164547.0,164547.0,169756.0,169756.0,169756.0,169756.0
1970Q3,66939.0,66939.0,66939.0,66939.0,66939.0,66939.0,66939.0,66956.0,66956.0,66956.0,...,166176.0,166176.0,166176.0,166176.0,166176.0,166176.0,171401.0,171401.0,171401.0,171401.0
1970Q4,67711.0,67711.0,67711.0,67711.0,67711.0,67711.0,67711.0,67729.0,67729.0,67729.0,...,167688.0,167688.0,167688.0,167688.0,167688.0,167688.0,172905.0,172905.0,172905.0,172905.0
1971Q1,66491.0,66491.0,66491.0,66491.0,66491.0,66491.0,66491.0,66508.0,66508.0,66508.0,...,166667.0,166667.0,166667.0,166667.0,166667.0,166667.0,171908.0,171908.0,171908.0,171908.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,450239.0,450485.0,451260.0,451260.0,451260.0,462139.0,462139.0,462139.0,461751.0
2016Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,453065.0,452886.0,464212.0,464212.0,464212.0,463678.0
2016Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,466997.0,466944.0,466790.0


,Final_Revised_RGDP,Final_Revised_RGDP_YoY_Growth,Real_Time_YoY_Growth,Acceleration Signal
Date,,,,
1972Q1,178900.0,4.067292,3.561384,1.468582
1972Q2,183665.0,4.985024,2.872323,0.608909
1972Q3,184154.0,3.454979,0.439940,-3.468096
1972Q4,187051.0,4.673195,3.031435,0.235730
1973Q1,196348.0,9.752935,10.321091,6.759707
...,...,...,...,...
2015Q2,457321.0,2.419158,2.640722,-0.485465
2015Q3,458708.0,1.903181,2.289849,-0.715573
2015Q4,461751.0,1.731693,1.878087,-0.817801
